## HAECO Data Report Collation

In [1]:
import pandas as pd
import numpy as np

#### Main Index file

In [2]:
file_name = '/Users/rahmanfahd/Downloads/Copy of HMA100-20210802.xlsx' 
alls = pd.read_excel(file_name, index_col=None)

In [3]:
alls.Status.value_counts()
alls1 = alls.loc[(alls['Status'] == 'ACTIVE')| (alls['Status'] == 'TERMINATING')]
alls1 = alls1[alls1['Home Section'].str.contains("^(?:0B|09|0G|0S|AC|CS|0JB|0J9|0JS|0JCS|0JNT)")|alls1['Current Section'].str.contains("^(?:0B|09|0G|0S|AC|CS|0JB|0J9|0JS|0JCS|0JNT)")]
alls1 = alls1.rename(columns={'Staff No': 'Staff No.'})

#alls.head(10)
#alls1.count()
#alls1['Staff No'].dtypes
#alls1['Staff No.'] = alls1['Staff No'].astype(str).str.zfill(6)

In [4]:
final = alls1[['Staff No.','Staff Name','Home Section']]

#### License data

In [5]:
file_name = '/Users/rahmanfahd/Downloads/QA_Lic_Full_Detailed_List-20220411.xlsx' 
lic = pd.read_excel(file_name, index_col=None)

In [6]:
lic = lic.iloc[6:]
lic.columns = lic.iloc[0]
lic = lic[1:]
lic = lic.iloc[1:]

lic['bl'] = lic['Module Properties-Module Title'].str.split('_').str[1]
lic['bl'] = lic['bl'].str.split('/').str[0]
lic['Basic License'] = np.where(lic['bl']=='FAA','FAA',lic['Module Properties-Module Title'].str.split('/').str[3])
lic['Basic License'] = np.where(((lic['bl']=='HKCAD')|(lic['bl']=='FAA')),lic['Basic License'], 'NDT' )
lic['Basic License'] = lic['Basic License'].replace(['B1.1'],'B1')
lic = lic[lic['Basic License'] != '--']

#lic['User Properties-Employee Number'].nunique()
lic2 = lic.groupby(['User Properties-Employee Number'])['Basic License'].apply('/'.join).reset_index()
lic2 = lic.groupby(['User Properties-Employee Number'])['Basic License'].apply('/ '.join).reset_index()
lic2['Basic License'] = lic2['Basic License'].str.split('/ ').apply(pd.unique)
lic2['Basic License'] = lic2['Basic License'].str.join('/ ')

In [7]:
lic2 = lic2.rename(columns={'User Properties-Employee Number': 'Staff No.'})
final = pd.merge(final, lic2, on="Staff No.", how='left' )

#### Authorization AC data

In [8]:
file_name = '/Users/rahmanfahd/Downloads/HCA114_Auth-HX-20210802.xlsx' 
auth = pd.read_excel(file_name, index_col=None)

In [9]:
def rename(df,column):
    df[column] = df[column].replace(['A332','A333','A332F','A333F','A339'],'A330')
    df[column] = df[column].replace(['A343','A342'],'A340')
    df[column] = df[column].replace(['B77W','B772','B773','B77LF','B77L'],'B777')
    df[column] = df[column].replace(['B763','B763F','B764','B762F','B762'],'B767')
    df[column] = df[column].replace(['A359','A35K'],'A350')
    df[column] = df[column].replace(['B789','B788','B78X'],'B787')
    df[column] = df[column].replace(['MD11F','MD11'],'MD11')
    df[column] = df[column].replace(['A319','A318','A321','A20N','A21N'],'A320')
    df[column] = df[column].replace(['B753','B752'],'B757')
    df[column] = df[column].replace(['B753','B752'],'B757')
    df[column] = df[column].replace(['A306','A306F'],'A300')
    df[column] = df[column].replace(['B738','B737','B739','B736'],'B737NG')
    df[column] = df[column].replace(['A388'],'A380')
    df[column] = df[column].replace(['B744F'],'B747-400')
    df[column] = df[column].replace(['B748I','B748F'],'B747-8')
    df[column] = df[column].replace(['B744'],'B747-Classic')
    df[column] = df[column].replace(['B747'],'B747-Classic')

In [10]:
auth['Auth AC'] = auth['Aircraft Type']
rename(auth,'Auth AC')
auth1 = auth.groupby(['Staff Name','Staff No.'])['Auth AC'].apply(', '.join).reset_index()
auth1['Auth AC'] = auth1['Auth AC'].str.split(', ').apply(pd.unique)
auth1['Auth AC'] = auth1['Auth AC'].str.join(', ')

In [11]:
#auth1 = auth1.rename(columns={'Staff No.': 'Staff No'})
final = pd.merge(final, auth1[['Staff No.','Auth AC']], on="Staff No.", how='left')

#### Authorization AC/Engine data

In [12]:
auth['Auth Eng'] = auth['Auth AC'] + "/" +auth['Engine Type']
auth2 = auth.groupby(['Staff Name','Staff No.'])['Auth Eng'].apply(', '.join).reset_index()
auth2['Auth Eng'] = auth2['Auth Eng'].str.split(', ').apply(pd.unique)
auth2['Auth Eng'] = auth2['Auth Eng'].str.join(', ')

In [13]:
#auth2 = auth2.rename(columns={'Staff No.': 'Staff No'})
final = pd.merge(final, auth2[['Staff No.','Auth Eng']],on="Staff No.", how='left' )

In [14]:
#final

,Staff No.,Staff Name,Home Section,Basic License,Auth AC,Auth Eng
0,74659,LAW HING CHOY,0BA4,NaN,"A330, B777","A330/T700, B777/--"
1,75540,FAN KWAI ON,CSDY,NaN,--,--/--
2,77554,TSE CHI BIU,0GMR,NaN,--,--/--
3,77687,KUEN SHUI BUN,0STY,NaN,--,--/--
4,77760,TAM CHING SHAN,CSNT,NaN,--,--/--
...,...,...,...,...,...,...
1557,342014,CHAN WAI KIU,0JSP,NaN,NaN,NaN
1558,342022,HUANG CHUN YAM,0JSP,NaN,NaN,NaN
1559,342030,LI PEI YI,0JSP,NaN,NaN,NaN
1560,342048,SO LING FUNG,0JSP,NaN,NaN,NaN


#### License AC/Engine data

In [15]:
lic['AC']=lic['Module Properties-Module Title'].str.split('/').str[1]
lic['AC'] = lic['AC'].str.split('_').str[0]
lic['Lic AC'] = np.where(lic['AC']!='B747-400',lic['AC'].str.split('-').str[0],'B747-400')
lic = lic.loc[lic['AC'] != '--']
rename(lic,'Lic AC')
lic1 = lic.groupby(['User Properties-Employee Number'])['Lic AC'].apply(', '.join).reset_index()
lic1['Lic AC'] = lic1['Lic AC'].str.split(', ').apply(pd.unique)
lic1['Lic AC']  = lic1['Lic AC'] .str.join(', ')

In [16]:
#lic1['Lic AC']

0                           B737NG, B747-400, B767, B777
1      A300, A310, A320, A330, A340, B747-Classic, B7...
2               B747-400, B747-Classic, B767, B777, B787
3         A320, A330, A340, B747-Classic, B747-400, B777
4      A320, A330, A340, B747-Classic, B747-400, B757...
                             ...                        
158    A300, A320, A330, A340, B737NG, B747-Classic, ...
159     A320, B737NG, B747-400, B747-Classic, B767, B777
160    A320, A330, A340, B737NG, B747-Classic, B747-4...
161       A330, A340, B747-Classic, B747-400, B767, B777
162             A330, A340, B747-Classic, B747-400, B777
Name: Lic AC, Length: 163, dtype: object

In [17]:
lic1 = lic1.rename(columns={'User Properties-Employee Number': 'Staff No.'})
final = pd.merge(final, lic1,  on="Staff No.", how='left' )

In [18]:
lic['engines']=lic['Module Properties-Module Title'].str.split('/').str[2]
lic = lic.loc[lic['engines'] != '--']
lic['Lic Eng'] = lic['Lic AC'] + "/" + lic['engines']
lic3 = lic.groupby(['User Properties-Employee Number'])['Lic Eng'].apply(', '.join).reset_index()
lic3['Lic Eng'] = lic3['Lic Eng'].str.split(', ').apply(pd.unique)
lic3['Lic Eng'] = lic3['Lic Eng'].str.join(', ')

In [19]:
lic3 = lic3.rename(columns={'User Properties-Employee Number': 'Staff No.'})
final = pd.merge(final, lic3,  on="Staff No.",how='left' )

#### Engrun data

In [20]:
file_name = '/Users/rahmanfahd/Downloads/QA_Auth_Full_Detailed_List-20220411.xlsx' 
auths = pd.read_excel(file_name, index_col=None)

In [21]:
auths = auths.iloc[6:]
auths.columns = auths.iloc[0]
auths = auths[1:]
auths = auths.iloc[1:]

auths['AC']=auths['Module Properties-Module ID'].str.split('_').str[0]
auths['Cat']=auths['Module Properties-Module ID'].str.split('_').str[2]

rename(auths,'AC')
auths["Engrun"] = np.where(auths['Cat']=='EGREXAMINER',auths['AC'] + "/" +auths['Module Properties-Module Title'].str.split("/").str[1],"")

auths4 = auths.groupby(['User Properties-Employee Number'])['Engrun'].apply(', '.join).reset_index()
auths4['Engrun'] = auths4['Engrun'].str.split(', ').apply(pd.unique)
auths4['Engrun'] = auths4['Engrun'].str.join(', ')

In [22]:
auths4 = auths4.rename(columns={'User Properties-Employee Number': 'Staff No.'})
final = pd.merge(final, auths4, on="Staff No.", how='left' )

#### Simulator Training Data

In [23]:
file_name = '/Users/rahmanfahd/Downloads/HCA114_Trng-EGRSmltr-20210802.xlsx' 
sim = pd.read_excel(file_name, index_col=None)

In [24]:
sim['AC Sim'] = sim['Course Title'].str.split(' ').str[0]
sim['Examinar'] = np.where(sim['Course Title'].str.contains('Exam'),sim['AC Sim'], 'False')
sim = sim[sim.groupby('Staff No.')['Course Completion Date'].transform('max') == sim['Course Completion Date']]

In [25]:
#sim = sim.rename(columns={'Staff No.': 'Staff No'})
final = pd.merge(final, sim[['Staff No.','AC Sim','Examinar']], on='Staff No.', how ='left')

#### EASA Course Data

In [26]:
file_name2 = '/Users/rahmanfahd/Downloads/HCA114_Trng-EASA-20210802.xlsx' 
easa_df = pd.read_excel(file_name2, index_col=None)

In [27]:
easa_df1 = easa_df.groupby(['Staff No.','Staff Name','Course Code'])['Course Title'].apply(', '.join).reset_index()
easa_df['Course Title'] = easa_df['Course Title'].replace({'EASA MOE & Company Procedure Training':'MOE', 'Aviation Legislation (M10) for EASA Part-66 Cat. B':'M10','Aviation Legislation Training for EASA Part-66 Cat. B':'M10','Human Factors (M9) for EASA Part-66 Cat. B':'M9','Human Factors (M9) for EASA Part-66 Cat. B (Multiple Choice)':'M9'})
easa_df1 = easa_df.groupby(['Staff No.','Staff Name'])['Course Title'].apply(', '.join).reset_index()
easa_df1['Course Title'] = easa_df1['Course Title'].str.split(', ').apply(pd.unique)
easa_df1['EASA'] = easa_df1['Course Title'].str.join(', ')
easa_df1['EASA'].apply(sorted);

In [28]:
#easa_df1 = easa_df1.rename(columns={'Staff No.': 'Staff No'})
final = pd.merge(final, easa_df1[['Staff No.','EASA']], on='Staff No.', how ='left')

#### Soft Skill data

In [29]:
file_name3 = '/Users/rahmanfahd/Downloads/HCA114_Trng-SoftSkill-20210802.xlsx' 
ss = pd.read_excel(file_name3, index_col=None)

In [30]:
#define values
values = ['N.H151','N.H150']

#drop rows that contain any value in the list
#ss = ss[ss.Course Code.isin(values) == False]
ss = ss[ss['Course Code'].isin(values) == False]
ss1 = ss.groupby(['Staff No.','Staff Name','Course Title'])['Course Code'].apply(', '.join).reset_index()
ss1 = ss.groupby(['Staff No.','Staff Name'])['Course Code'].apply(', '.join).reset_index()
ss1['Course Code'] = ss1['Course Code'].str.split(', ').apply(pd.unique)
ss1['Soft Skill Courses'] = ss1['Course Code'].str.join(', ')

In [31]:
final = pd.merge(final, ss1[['Staff No.','Soft Skill Courses']], on='Staff No.', how ='left')

#### TPA data

In [32]:
file_name4 = '/Users/rahmanfahd/Downloads/HCA114_Trng-TPA-20210802.xlsx' 
tpa = pd.read_excel(file_name4, index_col=None)

In [33]:
tpa1 = tpa.groupby(['Staff No.','Staff Name'])['Course Code'].apply(', '.join).reset_index()
tpa1 = tpa.groupby(['Staff No.','Staff Name'])['Course Code'].apply(', '.join).reset_index()
tpa1['Course Code'] = tpa1['Course Code'].str.split(', ').apply(pd.unique)
tpa1['Course Code'] = tpa1['Course Code'].str.join(', ')
tpa1 = tpa1.rename(columns={'Course Code': 'T.PA'})
tpa1['T.PA'].apply(sorted);

In [34]:
final = pd.merge(final, tpa1[['Staff No.','T.PA']], on='Staff No.', how ='left')

#### English data

In [35]:
file_name5 = '/Users/rahmanfahd/Downloads/HCA114_Trng-HK601A-20210802.xlsx' 
eng1 = pd.read_excel(file_name5, index_col=None)
eng1['Basic Eng'] = 'TradeEng'

In [36]:
file_name6 = '/Users/rahmanfahd/Downloads/HCA114_Trng-L21x-20210802.xlsx' 
eng2 = pd.read_excel(file_name6, index_col=None)

In [37]:
eng2['Courses'] =  eng2['Course Code'].str.split('.').str[1]
eng3 = eng2.groupby(['Staff No.','Staff Name'])['Courses'].apply(', '.join).reset_index()
eng3 = eng2.groupby(['Staff No.','Staff Name'])['Courses'].apply(', '.join).reset_index()
eng3['Course_Codes'] = eng3['Courses'].str.split(', ').apply(pd.unique)

eng3 = pd.merge(eng3, eng1[['Staff No.','Basic Eng']], on='Staff No.', how ='outer')
eng3['Basic Eng'] = eng3['Basic Eng'].replace(np.nan, "")
eng3['Courses'] = eng3['Courses'].replace(np.nan, "")

col         = 'Basic Eng'
conditions  = [ (eng3['Basic Eng'] == "") & (eng3['Courses'] != ""), (eng3['Basic Eng'] == "") & (eng3['Courses'] == ""), (eng3['Basic Eng'] != "") & (eng3['Courses'] != ""), (eng3['Basic Eng'] != "") & (eng3['Courses'] == "")]
choices     = [ eng3['Courses'], '', eng3['Courses'] + ", " + eng3['Basic Eng'], eng3['Basic Eng']]
    
eng3['English Training Completed'] = np.select(conditions, choices)
eng3 = eng3.dropna(subset=['Course_Codes'])
eng3['English Level'] = eng3.Course_Codes.apply(max)

In [38]:
final = pd.merge(final, eng3[['Staff No.','English Level','English Training Completed']], on='Staff No.', how ='left')

#### Sick Leave data

In [39]:
file_name7 = '/Users/rahmanfahd/Downloads/HML730-SL-P-20210802.xlsx' 
sl = pd.read_excel(file_name7, index_col=None)

In [40]:
#sl['Lv Type'].value_counts()
sl[['Start Date','End Date']] = sl[['Start Date','End Date']].apply(pd.to_datetime) #if conversion required
sl['days'] = (sl['End Date'] - sl['Start Date']).dt.days + 1
sl['all'] = np.where(sl['Lv Sess']=='2',0.5,0)
sl['total'] = sl['days']-sl['all']
sl['consec'] = np.where(sl['total'] != 1, True, False)

#sl['Lv Sess'].value_counts()
a = sl.groupby('Staff No',as_index=False).agg({'total':'sum'})
sls = sl.loc[sl['consec'] == False]
b = sls.groupby('Staff No',as_index=False).agg({'total':'sum'})

#a.loc[a['Staff No'] == 201749]
#b.loc[b['Staff No'] == 201749]

a = pd.merge(a, b, on="Staff No", how='left')
a['S/L'] = a['total_y'].astype(str) + " (" + a['total_x'].astype(str) + ") "

In [41]:
a = a.rename(columns={'Staff No': 'Staff No.'})
final = pd.merge(final, a[['Staff No.','S/L']], on='Staff No.', how ='left')

In [42]:
final['Staff No.'] = final['Staff No.'].astype(str).str.zfill(6)
final['S/L'] = final['S/L'] .fillna('0 (0)')

### Final Output

In [43]:
#pd.set_option('display.max_rows', 1500)
#final.head(1500)

#### Excel file name

In [44]:
#final.to_excel("output.xlsx")

In [46]:
#final.describe()

In [47]:
#final.loc[final['Staff No.'] == '311580']


,Staff No.,Staff Name,Home Section,Basic License,Auth AC,Auth Eng,Lic AC,Lic Eng,Engrun,AC Sim,Examinar,EASA,Soft Skill Courses,T.PA,English Level,English Training Completed,S/L
